In [1]:
import numpy as np
import pandas as pd

import sklearn
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from numpy import array
from numpy import hstack

from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
import tensorflow as tf

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
df = pd.read_csv("/content/drive/MyDrive/data_senegal/senegal_weather_spei_1970_2020_025x025_v3.csv")
df = df.dropna()
df = df.reset_index(drop=True)

scaler = StandardScaler()
data_scaled = scaler.fit_transform(df)
data_scaled_withcol = pd.DataFrame(data_scaled,columns = df.columns) 
data_scaled_withcol['year'] = df['year']
data_scaled_withcol['month'] = df['month']
data_scaled_withcol['gpslat'] = df['gpslat']
data_scaled_withcol['gpslon'] = df['gpslon']

In [22]:
in_seq_temp = array(df['temperature'])
#in_seq_humidex = array(data_scaled_withcol["humidex"])
in_seq_prcp = array(df["total_precipitation"])
out_seq_spei = array(df["spei_1mon"])

in_seq_temp = in_seq_temp.reshape((len(in_seq_temp), 1))
in_seq_prcp = in_seq_prcp.reshape((len(in_seq_prcp), 1))
#in_seq_humidex = in_seq_humidex.reshape((len(in_seq_humidex), 1))
out_seq_spei = out_seq_spei.reshape((len(out_seq_spei), 1))


dataset = hstack((in_seq_temp, in_seq_prcp, out_seq_spei))

n_steps = 12

In [23]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
  X, y = list(), list()
  for i in range(len(sequences)):
    # find the end of this pattern
    end_ix = i + n_steps
    # check if we are beyond the dataset
    if end_ix > len(sequences):
      break
    # gather input and output parts of the pattern
    seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
    X.append(seq_x)
    y.append(seq_y)
  return array(X), array(y)

In [24]:
X, y = split_sequences(dataset, n_steps)

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [26]:
X_train.shape

(147750, 12, 2)

In [27]:
n_features = X_train.shape[2]

model = Sequential()
model.add(LSTM(200, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(LSTM(200, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [28]:
with tf.device('/device:GPU:0'):
  model.fit(X_train, y_train, epochs=95, batch_size=64)

Epoch 1/95
2309/2309 [==============================] - 91s 39ms/step - loss: 0.9686
Epoch 2/95
2309/2309 [==============================] - 90s 39ms/step - loss: 0.4153
Epoch 3/95
2309/2309 [==============================] - 89s 39ms/step - loss: 0.3243
Epoch 4/95
2309/2309 [==============================] - 90s 39ms/step - loss: 0.2720
Epoch 5/95
2309/2309 [==============================] - 90s 39ms/step - loss: 0.2243
Epoch 6/95
2309/2309 [==============================] - 90s 39ms/step - loss: 0.1884
Epoch 7/95
2309/2309 [==============================] - 89s 39ms/step - loss: 0.1622
Epoch 8/95
2309/2309 [==============================] - 90s 39ms/step - loss: 0.1383
Epoch 9/95
2309/2309 [==============================] - 90s 39ms/step - loss: 0.1208
Epoch 10/95
2309/2309 [==============================] - 90s 39ms/step - loss: 0.1043
Epoch 11/95
2309/2309 [==============================] - 90s 39ms/step - loss: 0.0912
Epoch 12/95
2309/2309 [==============================] - 90s 39

In [29]:
y_hat = model.predict(X_test)
print(mean_squared_error(y_hat, y_test))
print(sklearn.metrics.r2_score(y_hat, y_test))

0.018384891270588635
0.9807274436450245
